In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import random
import pandas as pd
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.options.display.max_rows = 100
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
!nvidia-smi

Mon Jan 15 13:12:40 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.191.01   Driver Version: 450.191.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:1A:00.0 Off |                    0 |
| N/A   46C    P0    72W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:

random.seed(3)
np.random.seed(3)
seed = 3
batch_size = 1024

device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [5]:
device

'cuda'

In [6]:
!pwd

/root/.jupyter/connection


In [7]:
def to_categorical(y, num_classes=None, dtype='float32'):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1]==1 and len(input_shape)>1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()  # 拉成一维矩阵
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes, )
    categorical = np.reshape(categorical, output_shape)
    return categorical



In [8]:

def data_preparation():
    column_names = ['country_y', 'yearmonth', 'media_gained_decisiondate', 
            'media_total_decisiondate', 'followers_gained_decisiondate', 'followers_total_decisiondate', 'following_gained_decisiondate', 
            'following_total_decisiondate', 'media_gained_decisiondate_7days', 'media_total_decisiondate_7days', 'followers_gained_decisiondate_7days', 
            'followers_total_decisiondate_7days', 'following_gained_decisiondate_7days', 'following_total_decisiondate_7days', 'media_gained_decisiondate_15days', 
            'media_total_decisiondate_15days', 'followers_gained_decisiondate_15days', 'followers_total_decisiondate_15days', 'following_gained_decisiondate_15days', 
            'following_total_decisiondate_15days', 'media_gained_decisiondate_30days', 'media_total_decisiondate_30days', 'followers_gained_decisiondate_30days', 
            'followers_total_decisiondate_30days', 'following_gained_decisiondate_30days', 'following_total_decisiondate_30days', 'media_gained_decisiondate_60days', 
            'media_total_decisiondate_60days', 'followers_gained_decisiondate_60days', 'followers_total_decisiondate_60days', 'following_gained_decisiondate_60days', 
            'following_total_decisiondate_60days', 'media_gained_decisiondate_90days', 'media_total_decisiondate_90days', 'followers_gained_decisiondate_90days', 
            'followers_total_decisiondate_90days', 'following_gained_decisiondate_90days', 'following_total_decisiondate_90days', 'followers_rate_decisiondate_7days', 
            'following_rate_decisiondate_7days', 'media_rate_decisiondate_7days', 'followers_rate_decisiondate_15days', 'following_rate_decisiondate_15days', 
            'media_rate_decisiondate_15days', 'followers_rate_decisiondate_30days', 'following_rate_decisiondate_30days', 'media_rate_decisiondate_30days', 
            'followers_rate_decisiondate_60days', 'following_rate_decisiondate_60days', 'media_rate_decisiondate_60days', 'followers_rate_decisiondate_90days', 
            'following_rate_decisiondate_90days', 'media_rate_decisiondate_90days',  'avg_total_orders', 'avg_new_customers', 'avg_customer_num', 
            'first_campaign_time', 'new_influencer', 'last_orders_total', 'last_new_customers', 'last_customer_num', 'days_from_last_posting', 
            'days_from_last_sponsored', 'days_from_last_org_branded', 'days_from_last_org_nonbranded', 'days_interval_posting', 'days_interval_sponsored', 
            'days_interval_org_branded', 'days_interval_org_nonbranded', '90days_comment_count', '90days_like_count', 'num_sponsored_posts_90days', 
            'num_organic_branded_90days', 'num_organic_nonbranded_90days', 'num_posts_90days', '60days_comment_count', '60days_like_count', 
            'num_sponsored_posts_60days', 'num_organic_branded_60days', 'num_organic_nonbranded_60days', 'num_posts_60days', '30days_comment_count', 
            '30days_like_count', 'num_sponsored_posts_30days', 'num_organic_branded_30days', 'num_organic_nonbranded_30days', 'num_posts_30days', 
            '15days_comment_count', '15days_like_count', 'num_sponsored_posts_15days', 'num_organic_branded_15days', 'num_organic_nonbranded_15days', 
            'num_posts_15days', '7days_comment_count', '7days_like_count', 'num_sponsored_posts_7days', 'num_organic_branded_7days', 'num_organic_nonbranded_7days', 
            'num_posts_7days', 'accept', 'Revenue', 'reputation_change_v2']

    df = pd.read_csv('/var/tmp/code/recommendation_model/mmoe_model/data/precampaign_duringcampaign_features_acceptance_decision_performance_0502.csv', delimiter=',', index_col=None, low_memory=False)
    df = df[column_names]

#     train_df = pd.read_csv('/var/tmp/code/recommendation_model/mmoe_model/data/census-income.data.gz', delimiter=',', header=None, index_col=None, names=column_names)
#     test_df = pd.read_csv('/var/tmp/code/recommendation_model/mmoe_model/data/census-income.test.gz', delimiter=',', header=None, index_col=None, names=column_names)

    # 论文中第一组任务
    label_columns = ['accept', 'Revenue', 'reputation_change_v2']

    categorical_columns = ['first_campaign_time', 'country_y', 'yearmonth']

    df.fillna(0, inplace=True)
    df_transformed = pd.get_dummies(df.drop(label_columns, axis=1), columns=categorical_columns)
    df_labels = df[label_columns]


    return df_transformed, df_labels


In [9]:
df_transformed, df_labels = data_preparation()

In [10]:
df_transformed.iloc[0]

media_gained_decisiondate                  0.000000
media_total_decisiondate                  82.000000
followers_gained_decisiondate             -1.000000
followers_total_decisiondate            3334.000000
following_gained_decisiondate             -3.000000
following_total_decisiondate             687.000000
media_gained_decisiondate_7days           -1.000000
media_total_decisiondate_7days            86.000000
followers_gained_decisiondate_7days        3.000000
followers_total_decisiondate_7days      3330.000000
following_gained_decisiondate_7days        0.000000
following_total_decisiondate_7days       689.000000
media_gained_decisiondate_15days          -1.000000
media_total_decisiondate_15days           88.000000
followers_gained_decisiondate_15days      14.000000
followers_total_decisiondate_15days     3308.000000
following_gained_decisiondate_15days      -1.000000
following_total_decisiondate_15days      688.000000
media_gained_decisiondate_30days           0.000000
media_total_

In [11]:

def getTensorDataset(my_x, my_y):
    tensor_x = torch.Tensor(my_x)
    tensor_y = torch.Tensor(my_y)
    return torch.utils.data.TensorDataset(tensor_x, tensor_y)



In [12]:

class Expert(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(Expert, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        return out

class Tower(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(Tower, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

class MMOE(nn.Module):
    def __init__(self, input_size, num_experts, experts_out, experts_hidden, towers_hidden, tasks):
        super(MMOE, self).__init__()
        self.input_size = input_size
        self.num_experts = num_experts
        self.experts_out = experts_out
        self.experts_hidden = experts_hidden
        self.towers_hidden = towers_hidden
        self.tasks = tasks

        self.softmax = nn.Softmax(dim=1)

        self.experts = nn.ModuleList([Expert(self.input_size, self.experts_out, self.experts_hidden) for i in range(self.num_experts)])
        self.w_gates = nn.ParameterList([nn.Parameter(torch.randn(input_size, num_experts), requires_grad=True) for i in range(self.tasks)])
        self.towers = nn.ModuleList([Tower(self.experts_out, 1, self.towers_hidden) for i in range(self.tasks)])

    def forward(self, x):
        experts_o = [e(x) for e in self.experts]
        experts_o_tensor = torch.stack(experts_o)

        gates_o = [self.softmax(x @ g) for g in self.w_gates]

        tower_input = [g.t().unsqueeze(2).expand(-1, -1, self.experts_out) * experts_o_tensor for g in gates_o]
        tower_input = [torch.sum(ti, dim=0) for ti in tower_input]

        final_output = [t(ti) for t, ti in zip(self.towers, tower_input)]
        return final_output


In [13]:

def test(loader):
    t1_pred, t2_pred, t3_pred, t1_target, t2_target, t3_target = [], [], [], [], [], []
    model.eval()
    with torch.no_grad():
        epoch_loss = []
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            yhat = model(x)
            y1, y2, y3 = y[:, 0], y[:, 1], y[:, 2]
            yhat_1, yhat_2, yhat_3 = yhat[0], yhat[1], yhat[2]

            loss1 = bce_loss_fn(yhat_1, y1.view(-1, 1))
            loss2 = mse_loss_fn(yhat_2, y2.view(-1, 1))
            loss3 = mse_loss_fn(yhat_3, y3.view(-1, 1))
            loss = loss1 + loss2 + loss3

            t1_hat, t2_hat, t3_hat = list(yhat_1.cpu()), list(yhat_2.cpu()), list(yhat_3.cpu())

            t1_pred += t1_hat
            t2_pred += t2_hat
            t3_pred += t3_hat
            t1_target += list(y1.cpu())
            t2_target += list(y2.cpu())
            t3_target += list(y3.cpu())

    # t1_pred = [1 if x else 0 for x in list(t1_pred)]
    # t2_pred = [1 if x else 0 for x in list(t2_pred)]

    auc_1 = roc_auc_score(t1_target, t1_pred)
    # auc_2 = roc_auc_score(t2_target, t2_pred)
    # auc_3 = roc_auc_score(t3_target, t3_pred)
    return auc_1

In [14]:

# train_data, train_label, validation_data, validation_label, test_data, test_label, output_info = data_preparation()
#print(output_info)


In [15]:
df_labels_np = df_labels.to_numpy()

In [16]:
# df_label_tmp = np.column_stack((np.argmax(df_labels_np_tp[0]), np.argmax(df_labels_np_tp[1]), np.argmax(df_labels_np_tp[2])))
df_dataset = getTensorDataset(df_transformed.to_numpy(), df_labels_np)
train_size = int(0.8 * len(df_dataset))
test_size = len(df_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset=df_dataset, lengths=[train_size, test_size], generator=torch.manual_seed(0))

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

In [17]:
df_transformed.to_numpy().shape

(378961, 320)

In [18]:
len(train_loader)

297

In [19]:
len(test_loader)

75

In [20]:
model = MMOE(input_size=320, num_experts=6, experts_out=16, experts_hidden=32, towers_hidden=8, tasks=3)
model = model.to(device)
lr = 1e-4
n_epochs = 10
bce_loss_fn = nn.BCELoss(reduction='mean')
mse_loss_fn = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
losses = []
val_loss = []


In [21]:
for epoch in range(n_epochs):
    model.train()
    epoch_loss = []
    c = 0
    print("Epoch: {}/{}".format(epoch, n_epochs))
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        y_hat = model(x)

        y1, y2, y3 = y[:, 0], y[:, 1], y[:, 2]
        y_1, y_2, y_3 = y_hat[0], y_hat[1], y_hat[2]

        loss1 = bce_loss_fn(y_1, y1.view(-1, 1))
        loss2 = 0.000001 * mse_loss_fn(y_2, y2.view(-1, 1))
        loss3 = 0.00001 * mse_loss_fn(y_3, y3.view(-1, 1))
        
        loss = loss1 + loss2 + loss3
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss.append(loss.item())
        print('train loss1: {:.5f}, train loss2: {:.5f}, train loss3: {:.5f}'.format(loss1.item(), loss2.item(), loss3.item()))

    losses.append(np.mean(epoch_loss))

    # auc1 = test(train_loader)
    # print('train loss: {:.5f}, val task1 auc: {:.5f}, val task2 auc: {:.3f}, val task3 auc: {:.3f}'.format(np.mean(epoch_loss), auc1, auc2, auc3))

    # print('train loss: {:.5f}, val task1 auc: {:.5f}'.format(np.mean(epoch_loss), auc1))


Epoch: 0/10
train loss1: 17.30103, train loss2: 0.13399, train loss3: 1.76482
train loss1: 17.40749, train loss2: 0.11996, train loss3: 2.16972
train loss1: 18.95155, train loss2: 0.19151, train loss3: 2.59185
train loss1: 18.23562, train loss2: 0.04386, train loss3: 0.25947
train loss1: 16.43505, train loss2: 0.06076, train loss3: 1.69100
train loss1: 18.36349, train loss2: 0.37047, train loss3: 1.51856
train loss1: 17.30774, train loss2: 0.05799, train loss3: 0.85337
train loss1: 16.16909, train loss2: 0.24172, train loss3: 4.57718
train loss1: 17.91301, train loss2: 0.09147, train loss3: 0.96518
train loss1: 15.67210, train loss2: 0.49436, train loss3: 1.65566
train loss1: 14.61100, train loss2: 0.03607, train loss3: 2.14322
train loss1: 14.61942, train loss2: 3.51073, train loss3: 0.18444
train loss1: 16.26641, train loss2: 1.94085, train loss3: 0.54771
train loss1: 15.78852, train loss2: 0.05364, train loss3: 27.09285
train loss1: 16.92664, train loss2: 0.23138, train loss3: 0.152

In [22]:

auc1, auc2, auc3 = test(test_loader)
print('test task1 auc: {:.3f}, test task2 auc: {:.3f}, test task3 auc: {:.3f}'.format(auc1, auc2, auc3))

ValueError: not enough values to unpack (expected 6, got 4)